In [1]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np

In [2]:
raw_data = pd.read_csv('http://data.insideairbnb.com/united-states/ny/new-york-city/2020-07-07/data/listings.csv.gz',
                        compression='gzip')


C:\Users\guill\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (61,62,94) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [45]:
airbnb_data = raw_data


airbnb_data.iloc[:,30:50]

,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,street,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,city,state,zipcode,market,smart_location,country_code,country,latitude,longitude
0,https://a0.muscache.com/im/users/2259/profile_...,Washington Heights,0.0,0.0,['reviews'],t,f,"New York, NY, United States",Washington Heights,Washington Heights,Manhattan,New York,NY,10040,New York,"New York, NY",US,United States,40.85722,-73.93790
1,https://a0.muscache.com/im/pictures/user/50fc5...,Midtown,6.0,6.0,"['email', 'phone', 'reviews', 'offline_governm...",t,t,"New York, NY, United States",Manhattan,Midtown,Manhattan,New York,NY,10018,New York,"New York, NY",US,United States,40.75362,-73.98377
2,https://a0.muscache.com/im/users/4869/profile_...,Clinton Hill,1.0,1.0,"['email', 'phone', 'reviews', 'offline_governm...",t,t,"Brooklyn, NY, United States",Brooklyn,Clinton Hill,Brooklyn,Brooklyn,NY,11238,New York,"Brooklyn, NY",US,United States,40.68514,-73.95976
3,https://a0.muscache.com/im/users/7231/profile_...,Times Square,1.0,1.0,"['email', 'phone', 'reviews']",t,f,"New York, NY, United States",Manhattan,Hell's Kitchen,Manhattan,New York,NY,10019,New York,"New York, NY",US,United States,40.76551,-73.98631
4,https://a0.muscache.com/im/pictures/user/26745...,Flatiron District,1.0,1.0,"['email', 'phone', 'reviews', 'jumio', 'govern...",t,f,"New York, NY, United States",Midtown East,Murray Hill,Manhattan,New York,NY,10016,New York,"New York, NY",US,United States,40.74767,-73.97500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48583,https://a0.muscache.com/im/pictures/user/40d48...,Williamsburg,0.0,0.0,"['email', 'phone', 'offline_government_id', 's...",t,f,"New York, NY, United States",Manhattan,East Harlem,Manhattan,New York,NY,10029,New York,"New York, NY",US,United States,40.79319,-73.94360
48584,https://a0.muscache.com/im/users/2420592/profi...,DUMBO,1.0,1.0,"['email', 'phone', 'facebook', 'reviews', 'jum...",t,t,"Brooklyn, NY, United States",Brooklyn Navy Yard,Navy Yard,Brooklyn,Brooklyn,NY,11201,New York,"Brooklyn, NY",US,United States,40.70231,-73.98032
48585,https://a0.muscache.com/im/pictures/user/40d48...,Williamsburg,0.0,0.0,"['email', 'phone', 'offline_government_id', 's...",t,f,"Brooklyn, NY, United States",Brooklyn,Williamsburg,Brooklyn,Brooklyn,NY,11249,New York,"Brooklyn, NY",US,United States,40.71414,-73.96376
48586,https://a0.muscache.com/im/pictures/user/40d48...,Williamsburg,0.0,0.0,"['email', 'phone', 'offline_government_id', 's...",t,f,"Brooklyn, NY, United States",Brooklyn,Bushwick,Brooklyn,Brooklyn,NY,11206,New York,"Brooklyn, NY",US,United States,40.69850,-73.93186


We can separate our variables between categorical and numerical to apply different transformations to each type.

In [57]:
#Categorical variables to use
cat_vars = ["instant_bookable", "is_business_travel_ready", "cancellation_policy","host_is_superhost", "neighbourhood_cleansed","property_type","room_type", "bed_type"]
# Numerical Variables to use
num_vars = ['price', 'square_feet','minimum_nights','weekly_price', 'monthly_price', 'security_deposit','bathrooms', 'bedrooms', 'beds', "guests_included",
       'cleaning_fee', "reviews_per_month","host_listings_count",
           "calculated_host_listings_count_shared_rooms", "host_response_rate", "host_acceptance_rate",
           "review_scores_cleanliness","review_scores_value","review_scores_rating", "number_of_reviews"]

In [67]:
airbnb_data = raw_data

# Drop observation if categorical data is missing
before = airbnb_data.shape[0]
airbnb_data = airbnb_data.dropna(subset=cat_vars + ["number_of_reviews"])
airbnb_data = airbnb_data[airbnb_data['number_of_reviews']>0]
after = airbnb_data.shape[0]


airbnb_data = airbnb_data[cat_vars + num_vars]
print("Percent missing categorical data removed: {:.2%}".format((before-after)/before))

Percent missing categorical data removed: 23.39%


In [38]:
airbnb_data.columns

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'name', 'summary',
       'space', 'description', 'experiences_offered', 'neighborhood_overview',
       ...
       'instant_bookable', 'is_business_travel_ready', 'cancellation_policy',
       'require_guest_profile_picture', 'require_guest_phone_verification',
       'calculated_host_listings_count',
       'calculated_host_listings_count_entire_homes',
       'calculated_host_listings_count_private_rooms',
       'calculated_host_listings_count_shared_rooms', 'reviews_per_month'],
      dtype='object', length=106)

In [69]:
# Quick function to process currency and % numbers from strings to numerical floats
def clean_numerics(x):
    """ If the value is a string, then remove currency symbol and delimiters
    otherwise, the value is numeric and can be converted
    """
    if (x is np.nan) or (x=='NaN') or (x is None):
         x = np.nan

    elif isinstance(x, str):
        if '%' in x:
            clean_val = x.replace('%','')
            return (float(clean_val)/100)
        else:
            clean_val = x.replace('$', '').replace(',', '')
            return(  float(clean_val))
    return(x)


airbnb_data.loc[:,num_vars] = airbnb_data[num_vars].applymap(clean_numerics)



## Imputing

Missing values are always an issue when working with real data. We must handle them. We could drop all rows with missing values but then we wouldn't have much data left. We can impute the data, sklearn provides a simple imputer that can 'fill' in the missing values with different or even custom metrics. Here is an example of replacing the missing values with the median of the series.

In [70]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy="median")

# Num_vars is the list of
airbnb_num = airbnb_data[num_vars]

airbnb_num = imputer.fit_transform(airbnb_num)

In [65]:
from sklearn.base import BaseEstimator, TransformerMixin

class NewVariablesAdder(BaseEstimator, TransformerMixin):
    
    def __init__(self):
        pass
    
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        "review_scores_rating", "review_scores_cleanliness","review_scores_value", "number_of_reviews"
        ratings_over_reviews = X[:,-2]/X[:,-1]
        
        return np.c_[X, ratings_over_reviews]

## Encoding

Machine Learning algorithms don't work well with strings  or categorical data. We have to translate the categories to a numerical value. Let's convert different categories to numbers. We can use encoders to automate this task

In [24]:
airbnb_cat= airbnb_data[cat_vars]

In [25]:
from  sklearn.preprocessing import OrdinalEncoder
ordinal_encoder = OrdinalEncoder()

airbnb_cat_encoded = ordinal_encoder.fit_transform(airbnb_cat)
airbnb_cat_encoded[:,1:10]


array([[ 0.,  0.,  0., ..., 29.,  2.,  4.],
       [ 0.,  3.,  0., ...,  1.,  0.,  4.],
       [ 0.,  0.,  0., ..., 19.,  0.,  4.],
       ...,
       [ 0.,  0.,  0., ...,  1.,  2.,  4.],
       [ 0.,  0.,  0., ..., 23.,  0.,  4.],
       [ 0.,  3.,  0., ...,  1.,  2.,  4.]])

This looks great but we are not done yet. Machine learning algorithms can assume that two nearby values are more similar e.g 1 &2 vs 1&9 . This works when there is an order but is this a good idea with variables such as 'bed_type' , 'property_type' or 'room_type' ? 

We can transform our data to have a singular attribute per category. For Example, we create an attribute equal to 1 when the 'property_type' is 'House' and 0 otherwise. Repeat the same for all other categories.

This process is called one-hot encoding. If you come from a stats or econometrics background these are called dummy variables/attributes

Luckly for us, there is an encoder that performs this task efficiently.

In [26]:
from sklearn.preprocessing import OneHotEncoder

cat_encoder = OneHotEncoder()

airbnb_cat_hot_encoded = cat_encoder.fit_transform(airbnb_cat)
airbnb_cat_hot_encoded

<48563x281 sparse matrix of type '<class 'numpy.float64'>'
	with 388504 stored elements in Compressed Sparse Row format>

A wild sparse matrix appears!  Instead of a NumPy array the encoder returns a sparse matrix which is very convenient when we have several categorical attributes with hundreds of categories.

The matrix is full of 0 except for a single 1 per row. This would use a lot of memory to store but the spare matrix is smart and stors only the location of the nonzero elements.

# Feature Scaling

It is very important to scale our data when using machine learning algorithims. There are exceptions, but when the variables are in different scales we need to scale them to have the same scale.

A couple of ways to do it:

min-max scaling: substract the min value and divide by the range (max-min). Values will range from 0 to 1. The MinMaxScaler does this. 

standardization: substract mean and divide by standard deviation. You end up with 0 mean and unit variance. Values are not bounded in this case. Standardization is less affected by outliers. We can use StandardScaler.

In [27]:
from sklearn.preprocessing import StandardScaler

StandardScaler().fit_transform(airbnb_num)

array([[-0.14444458, -0.32826784, -0.07876442, ..., -0.11234708,
        -5.3530349 , -2.26102896],
       [ 0.13086405, -0.23906445,  4.63762052, ..., -0.11234708,
        -0.9502283 , -2.30436661],
       [-0.16867174, -0.32826784, -0.53401394, ..., -0.11234708,
         0.17869647,  0.4692433 ],
       ...,
       [-0.26558038,  0.96518127, -0.07876442, ..., -0.11234708,
        -1.96626059,  0.64259392],
       [-0.08938285,  0.96518127, -0.07876442, ..., -0.11234708,
        -1.96626059,  0.64259392],
       [-0.1928989 , -0.23906445, -0.07876442, ..., -0.11234708,
        -1.11956702, -1.39427586]])

In [51]:
test = NewVariablesAdder()
test.transform(airbnb_data)

array([[2060, 'https://www.airbnb.com/rooms/2060', 20200707031313, ...,
        0, 0.01, 80.0],
       [2595, 'https://www.airbnb.com/rooms/2595', 20200707031313, ...,
        0, 0.37, 1.9583333333333333],
       [3831, 'https://www.airbnb.com/rooms/3831', 20200707031313, ...,
        0, 4.65, 0.27217125382262997],
       ...,
       [44103612, 'https://www.airbnb.com/rooms/44103612',
        20200707031313, ..., 0, nan, nan],
       [44103854, 'https://www.airbnb.com/rooms/44103854',
        20200707031313, ..., 0, nan, nan],
       [44104534, 'https://www.airbnb.com/rooms/44104534',
        20200707031313, ..., 0, nan, nan]], dtype=object)

One data set can require several transformations which vary according the variable type. These must also be performed in the right order.

Scikit-Learn gifts us the Pipeline class to help with this ubiquitous process to write clean and efficient code.

In [71]:
from sklearn.pipeline import Pipeline

num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('add_variables', NewVariablesAdder()),
    ('std_scaler', StandardScaler())
])

num_transformed = num_pipeline.fit_transform(airbnb_num)

In [32]:
from sklearn.compose import ColumnTransformer


data_pipeline = ColumnTransformer([
    ('numerical', num_pipeline, num_vars),
    ('categorical', OneHotEncoder(), cat_vars),
    
])


airbnb_processed = data_pipeline.fit_transform(airbnb_data)

<9x9 sparse matrix of type '<class 'numpy.float64'>'
	with 81 stored elements in Compressed Sparse Row format>